In [ ]:
source('helpers.R')

In [ ]:
prepost <- readRDS("../../../ProcessedData/seurat/annotated_prepost_csf.rds")

In [ ]:
cts = c('NK','Monocyte','Macrophage')
target <- subset(prepost, cell_type %in% cts & Day %in% c('D0','D7'))

In [ ]:
target$prepost = 'Post'
target$prepost[target$Day == 'D0'] = 'Pre' 

In [ ]:
muscat.edgeR <- muscat.de.genes(
    seurat_obj = target, 
    cluster_col = 'cell_type', 
    group_col = 'prepost', 
    sample_col = 'sample_id', 
    method = 'edgeR'
)

In [ ]:
protein_coding <- filter_genes_by_biotype(muscat.edgeR$gene, biotype = "protein_coding")
muscat.edgeR <- muscat.edgeR[muscat.edgeR$gene %in%  c(protein_coding$filtered_genes, 'CAR'),]

In [ ]:
genes.up <- c(
  'SPON2','KLRF1', 'FCGR3A', "KIR2DL4", "KIR3DL1", "KIR3DL2",  "CX3CR1", "CXCR2","IL18RAP",
  'KLRG1', 'ZEB2','TBX21', 'S1PR5', 'GNLY', 'PRDM1', "AKR1C3", "PTGDS"
)
genes.down <- c('CST3','TOX2','CD200R1','DUSP4','ENTPD1','SRGAP2','ITGAE','LTB','ITGA1')

In [ ]:
ctype = "NK"
muscat.edgeR.ctype <- muscat.edgeR %>% 
    filter(cluster_id == ctype) %>% 
    dplyr::select(gene, p_val, logFC)

rownames(muscat.edgeR.ctype) <- muscat.edgeR.ctype$gene
colnames(muscat.edgeR.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

muscat.edgeR.ctype <- muscat.edgeR.ctype %>% dplyr::select(p_val_adj, avg_log2FC)
muscat.edgeR.ctype$avg_log2FC <- muscat.edgeR.ctype$avg_log2FC*-1

p <- do_VolcanoPlot(sample = target, genes.up = genes.up, genes.down = genes.down,
            de_genes = muscat.edgeR.ctype, pt.size=0.6,  min.segment.length=0.01, force=50, nudge_x=0.6,
            pval_cutoff = 0.05, FC_cutoff = log(2), n_genes = 15, order_tags_by='custom') + xlim(-7,12) +
    ylab(expression(paste("-", log["10"], "(P value)")))
p

In [ ]:
pdf(file.path(panel.path, paste0("4C_D0_vs_D7_",ctype,"_custom.pdf")), width=4, height=4)
p
dev.off()